# Naver Conversation of the Day

![Conversation of the Day](https://raw.githubusercontent.com/rareloto/workspace/master/naverdictionary-conversationoftheday-scraper/naver-conversationoftheday-20200815-cropped.png)

Hi! Welcome to my notebook.

I want to create a compilation of **Korean - English sentence pairs** from **Naver Dictionary's Conversation of the Day**.

At first I thought, I could type them out one by one, day by day to reinforce my Korean language learning in addition to creating a dataset for future projects.

But then, I remembered something called **'web scraping'**. Simply put, it is a technique of automatically extracting (*or scraping*) specific data you want from websites and saving them to a file or a database.

In this notebook, I'm going to take my first dip in web scraping in order to compile months, years of 'Conversation of the Day' from [Naver Dictionary](https://learn.dict.naver.com/conversation#/korean-en).

In [ ]:
import urllib.request
import re

url = 'https://learn.dict.naver.com/conversation#/korean-en/list'
html_contents = urllib.request.urlopen(url)
html_contents = str(html.read().decode('ms949')) # or maybe plain utf8 ?


**Find patterns in the html code around the data we want to extract**

![View source code](https://raw.githubusercontent.com/rareloto/workspace/master/naverdictionary-conversationoftheday-scraper/photos/view-page-source-find-patterns-koreng.png)

**Extract data from html:** *Korean - English sentence pairs, grammar, related words, date*

In [ ]:
# regex to extract sentence pairs
eng_sents = re.findall(r'<div class="txt_trans ng-binding" ng-show="transDisplay" ng-bind="item.trsl_sentence">(.+)</div>')
# how to strip html from text - jxb-bind-compiled-html binding ?
kor_sents = re.findall(r'<span class="u_word_dic" data-hook="tip" data-type="arken" data-lang="ko">(.+)</span>.</span></span>')
date = re.find(r'var regionDate = "[0-9]+"')

# extracting other data (e.g. grammar, related words)

In [ ]:
# append to list of conversations
texts = []

# loop through dates, extracting conversation per date

**Save to csv file**

In [ ]:
# save to dataframe
filename = f'{year_month}'
texts.to_csv(filename, index = False)

# References
* [K-MOOC: Python Web Scraping](http://blog.naver.com/PostView.nhn?blogId=powhy123&logNo=221193422772&categoryNo=19&parentCategoryNo=0&viewDate=&currentPage=1&postListTopCurrentPage=1&from=postView)